---
# **Flight Price Aanalysis Prediction**


---
#### **data description :**
---
``Airline`` - The name of the airline is stored in the "Airline" column. This is a categorical feature inherent in 6 different airlines.

``Flight`` - stores information about the flight code of the aircraft. This is a categorical feature.

``Source City`` - The city from which the flight departs. This is a categorical feature that has 6 unique cities.

``Departure Time`` - Departure time. This is a derived categorical feature obtained by grouping time periods by cells.

``Stops`` - The number of stops between the cities of departure and destination. Categorical attribute with 3 different values

``Arrival Time`` - Arrival time. This is a derived categorical feature created by grouping time intervals by cells. It has six different timestamps.

``Destination City`` - The city where the flight will land. This is a categorical feature that has 6 unique cities.

``Class`` - Passenger class; it has two different meanings: Business and Economy.

``Duration`` - The total amount of time spent on a trip between cities in hours.

``Days Left`` - Subtracting the travel date from the booking date. That is, for how many days was the ticket booked

``Price`` - Ticket price (Target variable)

---